# Домашнее задание: Расширение возможностей LLM через Tool Use

**Курс:** Валидация LLM  

## Цель задания

На лекции мы изучили, что базовые LLM имеют ограничения:
- Плохо справляются с точными вычислениями
- Не имеют доступа к актуальной информации
- Не могут взаимодействовать с внешними системами

В этом задании вы на практике увидите, как решается проблема вычислений с помощью Tool Use (Function Calling).

## Что мы сделаем

1. Создадим простую систему, где LLM может использовать калькулятор для точных вычислений
2. Покажем разницу между ответами с инструментом и без него
3. Вы добавите свою функцию в систему

## Используемые технологии

- **Python** для написания кода
- **LiteLLM** для унифицированного доступа к разным моделям
- **Ollama** с моделью Llama 3.2 (бесплатно, работает локально)

**Источники:**
- Документация LiteLLM: https://docs.litellm.ai/
- Ollama: https://ollama.ai/
- Статья про Function Calling: https://www.promptingguide.ai/applications/function_calling

## Установка необходимых библиотек

Для работы в Google Colab установим необходимые пакеты:

In [1]:
# Установка библиотек
!pip install litellm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 20.1 MB/s eta 0:00:00


## Настройка Ollama в Google Colab

Ollama позволяет запускать LLM модели локально. Установим Ollama и загрузим модель Llama 3.2 (небольшая модель, которая влезет в Colab):

In [4]:
!apt-get install zstd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (695 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...


In [5]:
# Установка Ollama
## установка инструмента для colab, чтобы можно было скачать ollama
!apt-get install zstd
## установка oolama (на красные комментарии можно не обращать внимание)
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [6]:
# Запуск Ollama в фоновом режиме
import subprocess
import time

# Запускаем сервер Ollama
process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)  # Даем время на запуск
print("Ollama сервер запущен")

Ollama сервер запущен


In [7]:
# Загрузка модели Llama 3.2 (1B параметров - легкая модель для Colab)
!ollama pull llama3.2:1b

## Часть 1: LLM без инструментов (baseline)

Сначала посмотрим, как базовая LLM справляется с математическими вычислениями без помощи калькулятора.

In [8]:
from litellm import completion

def ask_llm_without_tools(question):
    """
    Функция отправляет вопрос в LLM без использования инструментов.

    Параметры:
    - question: строка с вопросом для модели

    Возвращает:
    - ответ модели в виде строки
    """
    response = completion(
        model="ollama/llama3.2:1b",  # Используем локальную модель через Ollama
        messages=[{"role": "user", "content": question}],
        api_base="http://localhost:11434"  # Адрес локального Ollama сервера
    )
    return response.choices[0].message.content

# Пример: сложное умножение
question = "Сколько будет 8547 умножить на 37129?"
answer = ask_llm_without_tools(question)
print(f"Вопрос: {question}")
print(f"Ответ LLM: {answer}")
print(f"Правильный ответ: {8547 * 37129}")

Вопрос: Сколько будет 8547 умножить на 37129?
Ответ LLM: Чтобы найти результат умножения 8547 на 37129, мы можем использовать следующую формулу:

(8547 × 37129) = (8547 × 1000000 - 1)

Итак, 8547 умножилось на 37129, и результат составит 8547000000.
Правильный ответ: 317341563


**Что мы видим?**

Модель пытается ответить, но часто ошибается в сложных вычислениях. Это происходит потому, что LLM предсказывает следующий токен на основе паттернов, а не выполняет математические операции.

## Часть 2: Создание инструментов (Tools)

Теперь создадим функции-инструменты, которые LLM сможет использовать.

In [9]:
import json

def calculator(expression):
    """
    Функция выполняет математические вычисления.

    Параметры:
    - expression: строка с математическим выражением (например, "8547 * 37129")

    Возвращает:
    - результат вычисления
    """
    try:
        # Безопасное вычисление выражения
        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Ошибка вычисления: {str(e)}"

# Описание инструмента в формате, понятном для LLM
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Выполняет точные математические вычисления. Используй эту функцию для умножения, деления, сложения и вычитания чисел.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Математическое выражение для вычисления (например, '8547 * 37129')"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

print("Инструмент 'calculator' создан")

Инструмент 'calculator' создан


## Часть 3: LLM с инструментами

Теперь создадим систему, где LLM может решить, когда нужно использовать калькулятор.

In [10]:
def ask_llm_with_tools(question, tools):
    """
    Функция отправляет вопрос в LLM с возможностью использования инструментов.

    Параметры:
    - question: строка с вопросом
    - tools: список доступных инструментов

    Возвращает:
    - финальный ответ после использования инструментов
    """
    messages = [{"role": "user", "content": question}]

    # Первый запрос к LLM с информацией об инструментах
    response = completion(
        model="ollama/llama3.2:1b",
        messages=messages,
        tools=tools,
        api_base="http://localhost:11434"
    )

    response_message = response.choices[0].message

    # Проверяем, хочет ли модель использовать инструмент
    if hasattr(response_message, 'tool_calls') and response_message.tool_calls:
        # Модель решила использовать инструмент
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)

        print(f"LLM решила использовать инструмент: {function_name}")
        print(f"С параметрами: {arguments}")

        # Выполняем функцию
        if function_name == "calculator":
            result = calculator(arguments["expression"])
            print(f"Результат вычисления: {result}")

            # Отправляем результат обратно в LLM для формулировки ответа
            messages.append(response_message)
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": result
            })

            # Получаем финальный ответ
            final_response = completion(
                model="ollama/llama3.2:1b",
                messages=messages,
                api_base="http://localhost:11434"
            )
            return final_response.choices[0].message.content

    # Если инструменты не нужны, возвращаем обычный ответ
    return response_message.content

# Тестируем систему
question = "Сколько будет 8547 умножить на 37129?"
answer = ask_llm_with_tools(question, tools)
print(f"\nВопрос: {question}")
print(f"Финальный ответ: {answer}")

LLM решила использовать инструмент: calculator
С параметрами: {'expression': '8547 * 37129'}
Результат вычисления: 317341563

Вопрос: Сколько будет 8547 умножить на 37129?
Финальный ответ: Чтобы вычислить результат умножения 8547 на 37129, мы можем использовать тот же tool. Сначала мы преобразуем числа вfloating-pointные значения:

8547 = 8.547
37129 = 37.129

Теперь мы можем вычислить произведение:

8.547 * 37.129 ≈ 317.343

Следовательно, ответом на вопрос "Как many is 8547 умножить на 37129?" будет 317341563.


**Что произошло?**

1. LLM получила вопрос и описание доступных инструментов
2. Модель поняла, что нужен калькулятор, и вернула запрос на вызов функции
3. Наш код выполнил функцию calculator() и получил точный результат
4. Результат вернулся в LLM, которая сформулировала понятный ответ

Теперь вычисления точные!

## Часть 4: Сравнение результатов

Проверим разницу между подходами на нескольких примерах:

In [11]:
test_questions = [
    "Сколько будет 123 умножить на 456?",
    "Чему равно 9876 разделить на 12?",
    "Вычисли 2547 + 8965"
]

for q in test_questions:
    print("=" * 60)
    print(f"Вопрос: {q}")

    # Без инструментов
    answer_without = ask_llm_without_tools(q)
    print(f"Без инструментов: {answer_without}")

    # С инструментами
    answer_with = ask_llm_with_tools(q, tools)
    print(f"С инструментами: {answer_with}")
    print()

Вопрос: Сколько будет 123 умножить на 456?
Без инструментов: Чтобы найти результат этого произведения, мы можем использовать метод деления:

123 * 456 = 56,7724... 

Разделив 123 на 456, получим 0,27. 

Затем, сложив остаток и общий продукт, получим:

56 + 78 = 134
LLM решила использовать инструмент: calculator
С параметрами: {'expression': '123 * 456'}
Результат вычисления: 56088
С инструментами: Чтобы решить эту задачу, мы можем использовать функцию в Python, которая умножает two numbers together. 

В данном случае нам нужно использовать следующую функцию для умножения двух чисел: 

```python
def multiply(x, y):
    return x * y

result = multiply(123, 456)
print(result)  # Outputs: 56088
```

Это function создается по тому, что в вопросе имеется уникальная функция — calcualtor (которая может быть частью большего кода).

Вопрос: Чему равно 9876 разделить на 12?
Без инструментов: Возводя в квадрат и вычитая из него 1, получаем: 

9876^2 - 1 = 9767290. 

Затем мы делаем это по модулю 1

---

## Задание для самостоятельного выполнения

Ваша задача: **добавить новую функцию-инструмент в систему**.

### Что нужно сделать:

1. **Создайте новую функцию** `get_length()`, которая возвращает длину текстовой строки
2. **Добавьте описание этой функции** в список `tools` (по аналогии с калькулятором)
3. **Модифицируйте функцию** `ask_llm_with_tools()`, чтобы она могла вызывать вашу новую функцию
4. **Протестируйте** систему на вопросе: "Сколько символов в слове 'искусственный'?"

### Шаблон для выполнения:

In [ ]:
# Шаг 1: Создайте функцию get_length
def get_length(text):
    """
    Функция возвращает количество символов в тексте.

    Параметры:
    - text: строка текста

    Возвращает:
    - количество символов
    """
    # TODO: Ваш код здесь
    pass

# Шаг 2: Добавьте описание инструмента
tools_extended = [
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "Выполняет точные математические вычисления.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Математическое выражение"
                    }
                },
                "required": ["expression"]
            }
        }
    },
    # TODO: Добавьте описание функции get_length здесь
    # Структура аналогична calculator
]

# Шаг 3: Модифицируйте функцию ask_llm_with_tools
# Добавьте обработку вызова get_length в блоке if function_name == "calculator"

# Шаг 4: Протестируйте
# test_question = "Сколько символов в слове 'искусственный'?"
# answer = ask_llm_with_tools(test_question, tools_extended)
# print(answer)

---

## Критерии оценки

Ваше решение будет оцениваться по следующим критериям:

### 1. Корректность функции (4 балла)
- Функция `get_length()` корректно возвращает длину строки (2 балла)
- Функция обрабатывает edge cases (пустая строка, спецсимволы) (2 балла)

### 2. Описание инструмента (3 балла)
- Корректное JSON Schema описание функции в списке `tools` (1.5 балла)
- Понятное описание (description) того, что делает функция (1 балл)
- Правильное описание параметров (0.5 балла)

### 3. Интеграция с системой (2 балла)
- Модифицированная функция `ask_llm_with_tools()` корректно вызывает новый инструмент (1.5 баллов)
- Результат функции правильно передается обратно в LLM (0.5 балла)

### 4. Тестирование (1 балл)
- Приведен рабочий пример использования с вопросом про длину строки (0.5 балла)
- Результат работает корректно (0.5 балла)

---

## Формат сдачи

Сохраните выполненный ноутбук (файл .ipynb) с названием `ДЗ_1_Tool_Use_<Фамилия>_<ник в телеграмм>.ipynb` и пришлите в личные соощения телеграмм лектора Светланы Каримовой (@svet_ds).

**Успехов!**